# Tool Agent

In [ ]:
!pip install python-dotenv langchain-community wikipedia

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
model_path = os.getenv("MODEL_PATH")

In [ ]:
import logging

from app.llm.llamacpp.service import LlamaCppService

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

llm = LlamaCppService(model_path=model_path, n_gpu_layers=-1)

In [ ]:
from app.tools.langchain.wikipedia_query_run import LangchainWikipediaQueryRun
from app.agents.react.agent import ReActAgentFactory

tools = [LangchainWikipediaQueryRun(top_k_results=3, doc_content_chars_max=2048, exclude_from_scratch_pad=True)]
react_agent = ReActAgentFactory.start(llm=llm, tools=tools, step_limit=6)

In [ ]:
import json
from app.core.messages import Query

future = react_agent.ask(Query(goal="What year did the iphone come out?"))
response = future.get(timeout=120)
print(f"Steps: {json.dumps([step.model_dump() for step in response.metadata['steps']], indent=2)}")
print(f"Total tokens: {sum(step.token_usage.total_tokens for step in response.metadata['steps'])}")
print(response.final_output)

future = react_agent.ask(Query(goal="What other models exist?"))
response = future.get(timeout=120)
print(f"Steps: {json.dumps([step.model_dump() for step in response.metadata['steps']], indent=2)}")
print(f"Total tokens: {sum(step.token_usage.total_tokens for step in response.metadata['steps'])}")
print(response.final_output)

react_agent.stop()